# 1 Data Pre-processing

In [ ]:
# Loading libraries
import json
import pandas as pd
from shapely import MultiLineString, LineString
import kagglehub
import geopandas as gpd
import networkx as nx
from geopy.distance import great_circle
import matplotlib.pyplot as plt
import igraph as ig

In [ ]:
# Download latest version
path = kagglehub.dataset_download("sndorburian/underwater-marine-data-cables")
data = json.load(open("../data/kaggle/underwatercable.json"))
df_undersea = pd.DataFrame(data["features"])
df_undersea

In [ ]:
name_list = []
id_list = []
color_list = []

for i in df_undersea.index:
    name_i = df_undersea.loc[i].properties["name"]
    name_list.append(name_i)
    id_i = df_undersea.loc[i].properties["id"]
    id_list.append(id_i)
    color_i = df_undersea.loc[i].properties["color"]
    color_list.append(color_i)

geom_list = []
for i in df_undersea.index:
    geom_i = MultiLineString(pd.DataFrame(data["features"]).geometry.loc[i]["coordinates"])
    geom_list.append(geom_i)

gdf_undersea = df_undersea.copy(deep=False)
gdf_undersea = gpd.GeoDataFrame(gdf_undersea, geometry = geom_list)
gdf_undersea = gdf_undersea.set_crs(4326)

gdf_undersea["id"] = id_list
gdf_undersea["name"] = name_list
gdf_undersea["color"] = color_list

gdf_undersea

In [ ]:
gdf_undersea.explore()

# 2 Graph Network

## 2.1 Graph Creation & Analysis

In [ ]:
# Setting up an empty graph
G = nx.Graph()

In [ ]:
# Reading in the GeoJSON files of the edges and nodes as GeoPandas
gdf_nodes_json = gpd.read_file('../data/gdf_nodes.geojson')
gdf_edges_json = gpd.read_file('../data/gdf_edges.geojson')

In [ ]:
gdf_nodes_json

In [ ]:
gdf_edges_json

In [ ]:
# Creating graph G in NetworkX from the Geo DFs
G = nx.from_pandas_edgelist(
    gdf_edges_json,
    source='start_node',
    target='end_node',
    edge_attr='weight'
)

node_positions = {
    row['node_id']: (row.geometry.x, row.geometry.y)
    for index, row in gdf_nodes_json.iterrows()
}
nx.set_node_attributes(G, node_positions, name='pos')

In [ ]:
# Observing total number of edges and vertices in our graph
print(f"Number of Edges: {G.number_of_edges()}\nNumber of Vertices: {G.number_of_nodes()}")

# 2.2 Visualizing the Graph

In [ ]:
# Converting our NetworkX graph into an igraph so we can visualize it using Matplotlib
ig_graph = ig.Graph.from_networkx(G)

# Defining a dictionary for visual styles
visual_style = {
    "vertex_size": 10,
    "vertex_color": "orange",
    "vertex_label": None,
    "edge_width": 5,
    "edge_color": "#444444",
    "layout": "kamada_kawai",
}

# Creating a Matplotlib figure and axes
fig, ax = plt.subplots(figsize=(5, 5), dpi=150)

# Plotting the graph using igraph, but targeting the Matplotlib axes
ig.plot(
    ig_graph,
    target=ax,
    **visual_style
)

# Customizing and show the plot using Matplotlib commands
plt.axis('off')
plt.show()


## 2.3 Shortest Path with Djikstra

In [ ]:
# Assigning coordinates for our cities & countries of interest
PORTUGAL_COORDS = (38.736946, -9.142685)  # (lat, lon) for Lisbon, Portugal
BRAZIL_COORDS = (-3.731862, -38.526669)   # (lat, lon) for Fortaleza, Brazil 

In [ ]:
# Defining the function to find closest cable node to our cities of interest
def find_closest_node(graph, target_coords):

    target_lat, target_lon = target_coords
    closest_node_id = None
    min_dist = float('inf')

    # We iterate through all nodes and their data (attributes)
    for node_id, attributes in graph.nodes(data=True):
        node_lon, node_lat = attributes['pos']
        
        dist = great_circle((target_lat, target_lon), (node_lat, node_lon)).kilometers
        
        if dist < min_dist:
            min_dist = dist
            closest_node_id = node_id
            
    print(f"Found closest node for target {target_coords}: '{closest_node_id}' (Distance: {min_dist:.2f} km)")
    return closest_node_id

In [ ]:
# Defining the function for the Dijkstra algorithm
def dijkstra_sp(graph, source, target):
    
    # Calculating the shortest path using Djikstra algorithm
    path_length = nx.dijkstra_path_length(
        graph, source=source, target=target, weight='weight')

    # Get the actual sequence of nodes in the path
    path_nodes = nx.dijkstra_path(
        graph, source=source, target=target, weight='weight')

    print(f"Total distance: {path_length:,.2f} km")
    print(f"Path involves {len(path_nodes)} nodes.")
    return path_nodes

In [ ]:
# Identifying start (Portugal) and end (Brazil) nodes in the graph
start_node = find_closest_node(G, PORTUGAL_COORDS)
end_node = find_closest_node(G, BRAZIL_COORDS)

In [ ]:
shortest_path_nodes = dijkstra_sp(G, start_node, end_node)
shortest_path_nodes

In [ ]:
# Matching shortest path nodes to the additional data in the initial nodes DF
path_nodes_gdf = gdf_nodes_json.set_index('node_id').loc[shortest_path_nodes]

# Creating the path line
path_line = LineString(path_nodes_gdf.geometry)
path_line_gdf = gpd.GeoDataFrame(geometry=[path_line], crs="EPSG:4326")
m = path_line_gdf.explore(
    color="blue",
    style_kwds={'weight': 3},
    name="Shortest Path Line"
)

# Representing the individual path nodes
path_nodes_gdf.explore(
    m=m,
    marker_type='circle_marker',
    marker_kwds={'radius': 4, 'fill': True, 'color': 'green'},
    tooltip=['node_id', 'cable_id'],
    name="Path Nodes"
)

## 2.4 Second Shortest Path

In [ ]:
# Now, assuming the scenario that the shortest path undersea cable is destroyed, 
# what is the next shortest path / route from Portugal to Brazil?
G_copy = G.copy()
G_copy.remove_nodes_from(shortest_path_nodes)
G_copy.number_of_nodes()

In [ ]:
start_node = find_closest_node(G_copy, PORTUGAL_COORDS)
end_node = find_closest_node(G_copy, BRAZIL_COORDS)

In [ ]:
shortest_path_nodes = dijkstra_sp(G_copy, start_node, end_node)
shortest_path_nodes

In [ ]:
# Matching shortest path nodes to the additional data in the initial nodes DF
path_nodes_gdf = gdf_nodes_json.set_index('node_id').loc[shortest_path_nodes]

# Creating the path line
path_line = LineString(path_nodes_gdf.geometry)
path_line_gdf = gpd.GeoDataFrame(geometry=[path_line], crs="EPSG:4326")
m = path_line_gdf.explore(
    color="blue",
    style_kwds={'weight': 3},
    name="Shortest Path Line"
)

# Representing the individual path nodes
path_nodes_gdf.explore(
    m=m,
    marker_type='circle_marker',
    marker_kwds={'radius': 4, 'fill': True, 'color': 'green'},
    tooltip=['node_id', 'cable_id'],
    name="Path Nodes"
)

# 3 K-Shortest Paths

In [ ]:
# Assigning coordinates for our cities & countries of interest
PORTUGAL_COORDS = (38.736946, -9.142685)  # (lat, lon) for Lisbon, Portugal
BRAZIL_COORDS = (-3.731862, -38.526669)   # (lat, lon) for Fortaleza, Brazil 

In [ ]:
start_node = find_closest_node(G_copy, PORTUGAL_COORDS)
end_node = find_closest_node(G_copy, BRAZIL_COORDS)

In [ ]:
def k_shortest_paths(graph, source, target, k):

    # Get the generator for the k-shortest simple paths
    paths_generator = nx.shortest_simple_paths(graph, source=source, target=target, weight='weight')

    # Loop through the generator using a counter to get the first k paths
    path_count = 0
    
    final_paths = []
    for path in paths_generator:
        # First, check if we have already found the number of paths we want
        if path_count >= k:
            break

        # The path is valid, so we process it
        path_length = nx.path_weight(graph, path, weight='weight')
        
        # We use our own counter for the path number
        print(f"Path #{path_count + 1}:  (Nodes: {len(path)})  Length: {path_length:,.2f} km")

        # Finally, increment our counter
        path_count += 1

        final_paths.append(path)
        
    return final_paths


In [ ]:
k_shortest_path_nodes = k_shortest_paths(k=5, graph=G, source=start_node, target=end_node)

In [ ]:
# Matching shortest path nodes to the additional data in the initial nodes DF
path_nodes_gdf = gdf_nodes_json.set_index('node_id').loc[k_shortest_path_nodes[1]]

# Creating the path line
path_line = LineString(path_nodes_gdf.geometry)
path_line_gdf = gpd.GeoDataFrame(geometry=[path_line], crs="EPSG:4326")
m = path_line_gdf.explore(
    color="blue",
    style_kwds={'weight': 3},
    name="Shortest Path Line"
)

# Representing the individual path nodes
path_nodes_gdf.explore(
    m=m,
    marker_type='circle_marker',
    marker_kwds={'radius': 4, 'fill': True, 'color': 'green'},
    tooltip=['node_id', 'cable_id'],
    name="Path Nodes"
)

In [ ]:
gdf_nodes_json[gdf_nodes_json["cable_id"] == "ellalink"].node_id

# 4 Combining the Two

In [ ]:
# Assigning coordinates for our cities & countries of interest
PORTUGAL_COORDS = (38.736946, -9.142685)  # (lat, lon) for Lisbon, Portugal
BRAZIL_COORDS = (-3.731862, -38.526669)   # (lat, lon) for Fortaleza, Brazil 

In [ ]:
start_node = find_closest_node(G_copy, PORTUGAL_COORDS)
end_node = find_closest_node(G_copy, BRAZIL_COORDS)

In [ ]:
# Defining the function to find closest cable node to our cities of interest
def find_closest_node(graph, target_coords):

    target_lat, target_lon = target_coords
    closest_node_id = None
    min_dist = float('inf')

    # We iterate through all nodes and their data (attributes)
    for node_id, attributes in graph.nodes(data=True):
        node_lon, node_lat = attributes['pos']
        
        dist = great_circle((target_lat, target_lon), (node_lat, node_lon)).kilometers
        
        if dist < min_dist:
            min_dist = dist
            closest_node_id = node_id
            
    print(f"Found closest node for target {target_coords}: '{closest_node_id}' (Distance: {min_dist:.2f} km)")
    return closest_node_id

In [ ]:
def k_shortest_paths(graph, source, target, k):

    # Get the generator for the k-shortest simple paths
    paths_generator = nx.shortest_simple_paths(graph, source=source, target=target, weight='weight')

    # Loop through the generator using a counter to get the first k paths
    path_count = 0
    
    final_paths = []
    for path in paths_generator:
        # First, check if we have already found the number of paths we want
        if path_count >= k:
            break

        # The path is valid, so we process it
        path_length = nx.path_weight(graph, path, weight='weight')
        
        # We use our own counter for the path number
        print(f"Path #{path_count + 1}:  (Nodes: {len(path)})  Length: {path_length:,.2f} km")

        # Finally, increment our counter
        path_count += 1

        final_paths.append(path)
        
    return final_paths


In [ ]:
def shortest_paths(graph, source, target, algo='dijkstra', k=1, removal_line=None):

    working_graph = graph.copy()

    if removal_line:
        nodes_to_remove = gdf_nodes_json[gdf_nodes_json["cable_id"] == removal_line]['node_id'].tolist()
        print(f"Removed {len(nodes_to_remove)} nodes. The network is now damaged.")

        if nodes_to_remove:
            working_graph.remove_nodes_from(nodes_to_remove)

    if algo == 'dijkstra':
        return dijkstra_sp(working_graph, source, target)
        
    elif algo == 'yen':
        return k_shortest_paths(working_graph, source, target, k)

In [ ]:
# Example 1: Standard Dijkstra (removal_line is ignored)
print("--- RUNNING EXAMPLE 1 ---")
path1 = shortest_paths(G, start_node, end_node)

In [ ]:
# Example 2: Dijkstra AFTER removing the "ellalink" cable
print("\n--- RUNNING EXAMPLE 2 ---")
path2 = shortest_paths(G, start_node, end_node, removal_line="ellalink")

In [ ]:
# Example 3: Top 3 paths on the full network
print("\n--- RUNNING EXAMPLE 3 ---")
path3 = shortest_paths(G, start_node, end_node, algo='yen', k=3)

In [ ]:
# Example 4: Top 3 paths AFTER removing the "ellalink" cable
print("\n--- RUNNING EXAMPLE 4 ---")
path4 = shortest_paths(G, start_node, end_node, algo='yen', k=3, removal_line="ellalink")

In [ ]:
# Matching shortest path nodes to the additional data in the initial nodes DF
path_nodes_gdf = gdf_nodes_json.set_index('node_id').loc[path2]

# Creating the path line
path_line = LineString(path_nodes_gdf.geometry)
path_line_gdf = gpd.GeoDataFrame(geometry=[path_line], crs="EPSG:4326")
m = path_line_gdf.explore(
    color="blue",
    style_kwds={'weight': 3},
    name="Shortest Path Line"
)

# Representing the individual path nodes
path_nodes_gdf.explore(
    m=m,
    marker_type='circle_marker',
    marker_kwds={'radius': 4, 'fill': True, 'color': 'green'},
    tooltip=['node_id', 'cable_id'],
    name="Path Nodes"
)